# 05 - Reasoning Prompts

## Agentic Logistics Control System

This notebook documents prompt engineering for logistics reasoning:
- System prompts for domain expertise
- Situation assessment prompts
- Issue detection prompts
- Decision verification prompts

### Control Loop Role: REASONING Templates

```
OBSERVE -> [REASON] -> PLAN -> DECIDE -> ACT -> FEEDBACK -> (loop)
            ^
            |
      Prompts define HOW to reason
```

In [ ]:
# Setup
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from src.reasoning.prompts import PromptTemplates
from src.reasoning.grok_client import get_grok_client

print("Prompt templates loaded!")

## 1. System Prompts

In [ ]:
# Main logistics expert system prompt
print("=" * 50)
print("LOGISTICS EXPERT SYSTEM PROMPT")
print("=" * 50)
print(PromptTemplates.LOGISTICS_EXPERT_SYSTEM)

In [ ]:
# Situation assessment system prompt
print("=" * 50)
print("SITUATION ASSESSMENT SYSTEM PROMPT")
print("=" * 50)
print(PromptTemplates.SITUATION_ASSESSMENT_SYSTEM)

## 2. Situation Assessment Prompt

In [ ]:
# Sample data for testing prompts
sample_trucks = [
    {"id": "TRK-001", "name": "Alpha", "status": "en_route", "current_location": {"latitude": 40.71, "longitude": -74.00}},
    {"id": "TRK-002", "name": "Beta", "status": "en_route", "current_location": {"latitude": 40.75, "longitude": -73.98}},
    {"id": "TRK-003", "name": "Gamma", "status": "idle", "current_location": {"latitude": 40.68, "longitude": -74.04}},
    {"id": "TRK-004", "name": "Delta", "status": "stuck", "current_location": {"latitude": 40.72, "longitude": -73.99}},
]

sample_traffic = [
    {"segment_id": "SEG-I95-NB-1", "level": "heavy", "speed_kmh": 25, "incident_description": "Multi-vehicle accident"},
    {"segment_id": "SEG-9A-NB-1", "level": "light", "speed_kmh": 55, "incident_description": None},
    {"segment_id": "SEG-LOCAL-1", "level": "moderate", "speed_kmh": 35, "incident_description": None},
]

sample_loads = [
    {"id": "LOAD-001", "priority": "urgent", "description": "Medical supplies"},
    {"id": "LOAD-002", "priority": "high", "description": "Electronics"},
    {"id": "LOAD-003", "priority": "normal", "description": "General merchandise"},
]

sample_gps = [
    {"truck_id": "TRK-001", "location": {"latitude": 40.71, "longitude": -74.00}, "speed_kmh": 45},
    {"truck_id": "TRK-002", "location": {"latitude": 40.75, "longitude": -73.98}, "speed_kmh": 15},
    {"truck_id": "TRK-004", "location": {"latitude": 40.72, "longitude": -73.99}, "speed_kmh": 3},
]

print("Sample data created")

In [ ]:
# Format situation assessment prompt
situation_prompt = PromptTemplates.format_situation_assessment(
    trucks=sample_trucks,
    traffic=sample_traffic,
    loads=sample_loads,
    gps_readings=sample_gps
)

print("=" * 50)
print("FORMATTED SITUATION ASSESSMENT PROMPT")
print("=" * 50)
print(situation_prompt)

## 3. Issue Detection Prompt

In [ ]:
# Test truck data
test_truck = {
    "id": "TRK-004",
    "name": "Delta Freight",
    "status": "stuck",
    "current_location": {"latitude": 40.72, "longitude": -73.99},
    "fuel_level_percent": 35,
    "last_gps_reading": {
        "speed_kmh": 3,
        "heading": 45
    }
}

test_load = {
    "id": "LOAD-001",
    "description": "Medical supplies - time critical",
    "priority": "urgent",
    "delivery_deadline": "2024-01-15T14:00:00Z"
}

# Format issue detection prompt
issue_prompt = PromptTemplates.format_issue_detection(
    truck=test_truck,
    load=test_load,
    route=None,
    traffic=sample_traffic,
    speed_history=[45, 35, 20, 10, 5, 3, 3, 3, 3, 3]
)

print("=" * 50)
print("FORMATTED ISSUE DETECTION PROMPT")
print("=" * 50)
print(issue_prompt)

## 4. Traffic Impact Analysis Prompt

In [ ]:
# View the traffic impact analysis template
print("=" * 50)
print("TRAFFIC IMPACT ANALYSIS TEMPLATE")
print("=" * 50)
print(PromptTemplates.TRAFFIC_IMPACT_ANALYSIS.template)

## 5. Risk Evaluation Prompt

In [ ]:
# View the risk evaluation template
print("=" * 50)
print("RISK EVALUATION TEMPLATE")
print("=" * 50)
print(PromptTemplates.RISK_EVALUATION.template)

## 6. Multi-Issue Prioritization Prompt

In [ ]:
# View the prioritization template
print("=" * 50)
print("MULTI-ISSUE PRIORITIZATION TEMPLATE")
print("=" * 50)
print(PromptTemplates.MULTI_ISSUE_PRIORITIZATION.template)

## 7. Decision Verification Prompt

In [ ]:
# View the decision verification template
print("=" * 50)
print("DECISION VERIFICATION TEMPLATE")
print("=" * 50)
print(PromptTemplates.DECISION_VERIFICATION.template)

## 8. Explanation Generation Prompt

In [ ]:
# View the explanation generation template
print("=" * 50)
print("EXPLANATION GENERATION TEMPLATE")
print("=" * 50)
print(PromptTemplates.GENERATE_EXPLANATION.template)

## 9. Live Testing with Grok

In [ ]:
# Test prompts with Grok client
client = get_grok_client()

print(f"Testing with Grok (available: {client.is_available})")

# Test situation assessment
response = client.complete_json(
    prompt=situation_prompt,
    system_prompt=PromptTemplates.SITUATION_ASSESSMENT_SYSTEM
)

print("\n" + "=" * 50)
print("SITUATION ASSESSMENT RESPONSE")
print("=" * 50)
print(f"Success: {response['success']}")

if response.get('parsed'):
    import json
    print(json.dumps(response['parsed'], indent=2)[:1000])
else:
    print(response['content'][:1000])

In [ ]:
# Test issue detection
response = client.complete_json(
    prompt=issue_prompt,
    system_prompt=PromptTemplates.LOGISTICS_EXPERT_SYSTEM
)

print("=" * 50)
print("ISSUE DETECTION RESPONSE")
print("=" * 50)
print(f"Success: {response['success']}")

if response.get('parsed'):
    import json
    print(json.dumps(response['parsed'], indent=2))
else:
    print(response['content'][:1000])

## 10. Prompt Engineering Guidelines

### Key Principles Used:

1. **Structured Output**: Request JSON format with specific schema
2. **Domain Context**: System prompts establish logistics expertise
3. **Data Formatting**: Present data in clear, organized sections
4. **Explicit Instructions**: Tell the model exactly what to analyze
5. **Confidence Scores**: Request confidence levels for uncertainty handling
6. **Reasoning Traces**: Ask for step-by-step analysis

### Best Practices:

- Keep prompts focused on one task
- Include all relevant data in the prompt
- Use consistent JSON schemas
- Handle fallback cases gracefully

## Next Steps

1. Proceed to `06_reasoning_layer.ipynb` for complete reasoning implementation
2. These prompts will be used by the reasoning node in the control loop